In [1]:
import numpy as np
from seisLM.data_pipeline.foreshock_aftershock_dataset import (
  create_foreshock_aftershock_datasets
)
from seisLM.data_pipeline.foreshock_aftershock_dataloaders import (
  prepare_foreshock_aftershock_dataloaders)
import pandas as pd
import matplotlib.pyplot as plt
from seisLM.utils.project_path import gitdir

/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
import ml_collections
import torch
from seisLM.model.task_specific.foreshock_aftershock_models import (
  Conv1DShockClassifierLit, Wav2vec2ShockClassifierLit)


NUM_CLASSES = 4

def get_seisLM_config():

  config_path = gitdir() + '/seisLM/configs/foreshock_aftershock/seisLM_shock_classifier.json'
  with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
  config = ml_collections.ConfigDict(config)

  config.model_args.num_classes = NUM_CLASSES
  return config


def get_conv1d_config():
  config_path = gitdir() + '/seisLM/configs/foreshock_aftershock/conv1d_shock_classifier.json'
  with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
  config = ml_collections.ConfigDict(config)
  config.model_args.num_classes = NUM_CLASSES
  return config



seisLM_config = get_seisLM_config()
conv1d_config = get_conv1d_config()

models = {}

models['seisLM'] = Wav2vec2ShockClassifierLit(
  model_config=seisLM_config.model_args,
  training_config=seisLM_config.trainer_args,
  ).model


models['cnn'] = Conv1DShockClassifierLit(
  model_config=conv1d_config.model_args,
  training_config=conv1d_config.trainer_args,
  ).model


loaders = prepare_foreshock_aftershock_dataloaders(
  num_classes=seisLM_config.model_args.num_classes,
  **seisLM_config.data_args
)


Seed set to 42
Seed set to 42


In [3]:
model = models['seisLM']
device = 'cpu'
model = model.to(device)

waveforms, _ = next(iter(loaders['test']))

sample_idx = 0
num_channels_to_display = 10

with torch.no_grad():
  model(waveforms.to(device))
  # waveforms_amp_peak_norm = waveforms / (
  #   torch.max(torch.abs(waveforms), dim=-1, keepdim=True).values + 1e-10
  # )

  # wav2vec2_outputs = model.wav2vec2(
  #   waveforms_amp_peak_norm.to(device),
  #   attention_mask=None,
  #   output_attentions=False,
  #   output_hidden_states=True,
  # )

  # wav2vec2_outputs = model.get_wav2vec2_hidden_states(
  #   waveforms.to(device),
  #   concat_downsampled_input=seisLM_config.model_args.concat_downsampled_input
  # )
  # convolved_features = model.wav2vec2(waveforms_amp_peak_norm.to(device))

# fig, axes = plt.subplots(3, 1, figsize=(10, 10))

# axes[0].plot(waveforms_amp_peak_norm[0].T)
# axes[0].set_title('Waveform')

# axes[1].plot(
#   wav2vec2_outputs.extract_features[0, :num_channels_to_display, :].detach().cpu().numpy().T)

# axes[2].plot(
#   wav2vec2_outputs.last_hidden_state[0, :num_channels_to_display, :].detach().cpu().numpy().T
# )


# axes[2].plot(wav2vec2_outputs[0, :, :num_channels_to_display].detach().cpu().numpy())
# axes[2].set_title('Wav2Vec2 hidden states')


hidden-states has nan? tensor(True)


In [4]:
from sklearn.metrics import accuracy_score


for key, model in models.items():
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = models[key].to(device)

  all_gt_class = []
  all_pred_class = []
  all_losses = []

  for X, y in loaders['test']:
    with torch.no_grad():
      logits = model(X.to(device)).cpu()
      pred_class = np.argmax(logits.numpy(), axis=1)
      loss = torch.nn.functional.cross_entropy(logits, y)

    all_gt_class.append(y)
    all_pred_class.append(pred_class)
    all_losses.append(loss)

  all_gt_class = np.concatenate(all_gt_class)
  all_pred_class = np.concatenate(all_pred_class)
  all_losses = np.array(all_losses)

  print(f'model {key} acc at init', accuracy_score(all_gt_class, all_pred_class))
  print(f'model {key} loss at init', np.mean(all_losses))

hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tensor(False, device='cuda:0')
hidden-states has nan? tenso

In [5]:
wav2vec2_outputs[0, :, -num_channels_to_display:].shape

NameError: name 'wav2vec2_outputs' is not defined

In [ ]:
waveforms.shape

torch.Size([64, 3, 2500])

: 